In [2]:
import os
import argparse

import polars as pl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
import json
from pathlib import Path
import numpy as np

In [3]:
os.chdir("/Users/yiyichen/PycharmProjects/MRL/")

In [4]:
from preprocessor.preprocessing import preprocessing_one_tweet

In [5]:
# lang: fr, nl,it, el, pl, de, es, fi

In [6]:
dataset_path = "datasets/hate_speech_detection"

In [7]:
def split_data(df):
    train, test_val = train_test_split(df, test_size=0.2, random_state=1, shuffle=True)
    val, test = train_test_split(test_val, test_size=0.5,random_state=1, shuffle=True)
    return train, val, test

# Finnish

In [65]:
data_path = os.path.join(dataset_path, "finnish")

In [76]:
lexica = pd.read_csv(os.path.join(data_path, "hurtlex_FI.tsv"), sep="\t")

In [77]:
lemmas = lexica["lemma"].to_list()

In [67]:
df_ls =[]
for file in os.listdir(os.path.join(data_path, "fi-xed")):
    print(file)
    filepath = os.path.join(data_path, "fi-xed", file)
    df_ls.append(pd.read_csv(filepath))

val.csv
test.csv
train.csv


In [68]:
df = pd.concat(df_ls)

In [70]:
label_map = {
    "emotion": {"1": "ANGER", "2": "ANTICIP.", "3": "DISGUST", "4": "FEAR", "5": "JOY", "6": "SADNESS", "7": "SURPRISE",
                "8": "TRUST"},
    "sentiment": {"1": "negative", "2": "positive", "3": "negative", "4": "negative", "5": "positive", "6": "negative",
                  "7": "unknown", "8": "positive"}
}

In [104]:
df_neutral = read_txt(os.path.join(data_path, "fi-xed", "neu_fi.txt"), "fi")

In [106]:
df_neutral["label"]= [0 for x in range(len(df_neutral))]

In [109]:
df_neutral=df_neutral[["text", "label"]]

In [74]:
df_fi = read_data(os.path.join(data_path, "fi-xed", "fi-annotated.tsv"), "fi")

after dedup,  11068
negative    5615
positive    4400
Name: sentiment, dtype: int64


In [81]:
df_fi_neg.loc[df_fi_neg["text"].str.contains("|".join(lemmas)), "hsd"]= 1

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [83]:
df_fi_neg.hsd.value_counts()

1.0    1984
Name: hsd, dtype: int64

In [91]:
df_hate = df_fi_neg[df_fi_neg["hsd"]==1]

In [84]:
df_fi_pos = df_fi[df_fi["sentiment"]=="positive"]

In [90]:
df_fi_pos = df_fi_pos.sample(1984)

In [92]:
df_fi_pos["hsd"] = [0 for x in range(len(df_fi_pos))]

In [110]:
df_fi_pos["label"] =  [0 for x in range(len(df_fi_pos))]
df_fi_pos = df_fi_pos[["text", "label"]]
df_normal = pd.concat([df_fi_pos, df_neutral])

In [112]:
df_normal = df_normal.sample(1984)

In [120]:
df_hate["label"] = [1 for x in range(len(df_hate))]
df_hate = df_hate[["text", "label"]]

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_62920/2158479312.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hate["label"] = [1 for x in range(len(df_hate))]


In [121]:
df = pd.concat([df_normal, df_hate])

In [122]:
df

,text,label
11308,"On mukavaa, kun on joku, jonka voi hyvästellä.",0
4234,Löysimme hänet.,0
10782,"Meidän suhteessamme oli koko ajan naurua, naur...",0
8145,Toisen täytyy työntää Loomisia.,0
6569,Lähdemme ennen sitä.,0
...,...,...
12684,Sinun on varmasti nälkä.,1
12694,Tämä satuttaa sydäntäni.,1
12701,Et jää tänne etkä ota minua mukaasi.,1
12708,"Hän vaivalloisesti edes muisti isäänsä, joka m...",1


In [96]:
# df.rename(columns={"hsd":"label"}, inplace=True)

In [97]:
# df = df[["text", "label"]]

In [123]:
df.label.value_counts()


0    1984
1    1984
Name: label, dtype: int64

In [124]:
df.LEN= df.text.str.len()
min(df.LEN)

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_62920/428466479.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.LEN= df.text.str.len()


3

In [125]:
train, val, test = split_data(df)

In [127]:
train.to_csv(os.path.join(data_path, "preprocessed", "train.csv"))
val.to_csv(os.path.join(data_path, "preprocessed", "val.csv"))
test.to_csv(os.path.join(data_path, "preprocessed", "test.csv"))


In [103]:
def read_txt(filepath, lang):
    texts = []
    with open(filepath) as f:
        for line in f.readlines():
            if line.split("\t"):
                if lang == "fi":
                    _, text, _, _, _ = line.split('\t')
                else:
                    _, text = line.split("\t")
                texts.append(text.replace('\n', ''))
    neus = ["neutral" for _ in range(len(texts))]
    df = pd.DataFrame.from_dict({"text": texts, "sentiment": neus})
    df.drop_duplicates(subset="text", inplace=True)
    return df

In [71]:

def read_data(data_path, lang):
    # load_data
    df = pd.read_csv(data_path, sep='\t', header=None).rename({
        0: "text", 1: "emotions"
    }, axis=1)

    df["emotions"] = df["emotions"].str.replace(" ", "")

    # Create one vs all Encoding
    for i in range(1, 9):
        df[str(i)] = df["emotions"].str.contains(str(i)).astype(int).replace(1, label_map["sentiment"][str(i)])
        df[label_map["emotion"][str(i)]] = df["emotions"].str.contains(str(i)).astype(int)

    # Calculate number of positive and number of negative samples in the dataset
    df["n_positive"] = np.sum((df.iloc[:, 0:18] == "positive"), axis=1)
    df["n_negative"] = np.sum((df.iloc[:, 0:18] == "negative"), axis=1)
    df = df.drop([str(i) for i in range(1, 9)], axis=1)
    
    # How many annotations are there per sample?
    df["n_annots"] = np.sum(df.iloc[:, 2:10], axis=1)
    if lang in ["en", "fi"]:
        
        # Determine if the annotation is more positive or negative
        df = df[df['n_annots'] == 1]

        df.loc[df['n_negative'] == 1, "sentiment"] = "negative"
        df.loc[df['n_positive'] == 1, "sentiment"] = "positive"
    
    df.drop_duplicates(subset='text', inplace=True)
    print('after dedup, ', len(df))
    print(df.sentiment.value_counts())
    # return df[['text', 'sentiment']]
    return df

# Spanish

## AMI 2018

In [40]:
data_path = os.path.join(dataset_path, "spanish")

In [41]:
df_ami_2018 = pd.read_csv(os.path.join(data_path, "AMI2018", "es_AMI_TrainingSet.csv"), sep=";")

In [42]:
df_ami_2018 = df_ami_2018[["tweet", "misogynous"]]

In [43]:
df_ami_2018.loc[df_ami_2018["misogynous"]==1, "label"]=1
df_ami_2018.loc[df_ami_2018["misogynous"]==0, "label"]=0

In [44]:
df_ami_2018.label.value_counts()

0.0    1658
1.0    1649
Name: label, dtype: int64

In [45]:
df_ls = []
for file in os.listdir(os.path.join(data_path, "hateval2019")):
    filepath = os.path.join(data_path, "hateval2019", file)
    df_ = pd.read_csv(filepath)
    df_ls.append(df_)

In [46]:
df_hateval = pd.concat(df_ls)

In [47]:
df_hateval.loc[df_hateval["HS"]==1, "label"]=1
df_hateval.loc[df_hateval["HS"]==0, "label"]=0

In [48]:
df_hateval.label.value_counts()

0.0    11391
1.0     8209
Name: label, dtype: int64

In [49]:
df_hateval = df_hateval[["text", "label"]]

In [50]:
df_ami_2018.rename(columns={"tweet":"text"}, inplace=True)

In [51]:
df_ami_2018 = df_ami_2018[["text", "label"]]

In [52]:
df_ami_2018.label.value_counts()

0.0    1658
1.0    1649
Name: label, dtype: int64

In [53]:
df = pd.concat([df_ami_2018, df_hateval])

In [54]:
df.text = df.text.apply(preprocessing_one_tweet)

In [55]:
df["LEN"] = df.text.str.len()

In [56]:
df= df[df["LEN"]>=10]

In [57]:
df.label.value_counts()

0.0    13030
1.0     9858
Name: label, dtype: int64

In [59]:
df.drop_duplicates(subset=["text"], inplace=True)

In [60]:
df.label.value_counts()

0.0    11505
1.0     8511
Name: label, dtype: int64

In [61]:
df = df.groupby("label").sample(8511)

In [62]:
train, val, test = split_data(df)

In [63]:
train.label.value_counts()

0.0    6832
1.0    6785
Name: label, dtype: int64

In [64]:
train.to_csv(os.path.join(data_path, "preprocessed", "train.csv"))
val.to_csv(os.path.join(data_path, "preprocessed", "val.csv"))
test.to_csv(os.path.join(data_path, "preprocessed", "test.csv"))

# German

## GermEval-2018

In [11]:
data_path = os.path.join(dataset_path, "german",  "GermEval-2018")

In [14]:
df_test = pd.read_csv(os.path.join(data_path, "germeval2018.test.txt"), sep="\t", header=None)

In [19]:
df_test.rename(columns={1:"OFFN", 2:"TYPE"}, inplace=True)

In [20]:
df_test["TYPE"].value_counts()

OTHER        2248
ABUSE         737
INSULT        368
PROFANITY      45
Name: TYPE, dtype: int64

In [21]:
df_train = pd.read_csv(os.path.join(data_path, "germeval2018.training.txt"), sep="\t", header=None)

In [22]:
df_train.rename(columns={1:"OFFN", 2:"TYPE"}, inplace=True)

In [24]:
df_train.OFFN.value_counts()

OTHER      3321
OFFENSE    1688
Name: OFFN, dtype: int64

In [25]:
df_germeval = pd.concat([df_test, df_train])

In [27]:
df_germeval.rename(columns= {0:"text"}, inplace=True)

In [29]:
df_germeval.loc[df_germeval["OFFN"]=="OFFENSE", "label"]=1
df_germeval.loc[df_germeval["OFFN"]=="OTHER", "label"]=0

In [31]:
df_germeval.label.value_counts()

0.0    5569
1.0    2838
Name: label, dtype: int64

In [111]:
df_germeval

,text,OFFN,TYPE,label
0,"Meine Mutter hat mir erzählt, dass mein Vater ...",OTHER,OTHER,0.0
1,@Tom174_ @davidbest95 Meine Reaktion; |LBR| Ni...,OTHER,OTHER,0.0
2,"#Merkel rollt dem Emir von #Katar, der islamis...",OTHER,OTHER,0.0
3,„Merle ist kein junges unschuldiges Mädchen“ K...,OTHER,OTHER,0.0
4,@umweltundaktiv Asylantenflut bringt eben nur ...,OFFENSE,ABUSE,1.0
...,...,...,...,...
5004,Gegens. Zul. zu Patenamt &amp; gegenseitige An...,OTHER,OTHER,0.0
5005,"@GlasenappHenrik Zu Merkel fällt mir nur ein, ...",OFFENSE,INSULT,1.0
5006,@KokoLores20 @krippmarie Ein richtiges Zeichen...,OFFENSE,ABUSE,1.0
5007,"@Hartes_Geld ,Honecker‘Merkel macht uns zur ,D...",OFFENSE,ABUSE,1.0


## fb_hate_speech

In [32]:
data_path = os.path.join(dataset_path, "german",  "fb_hate_speech_csv-DE")

In [33]:
df_fb = pd.read_csv(os.path.join(data_path, "comments.csv"))

In [35]:
df_tags = pd.read_csv(os.path.join(data_path, "annotated_comments.csv"))

In [37]:
df_tags.valence.value_counts()

2    843
1    488
Name: valence, dtype: int64

In [40]:
df_fb_tag = pd.merge(df_fb, df_tags, on="comment_id")

In [43]:
df_fb_tag = df_fb_tag[["message", "valence"]]

In [47]:
df_fb_tag.drop_duplicates(subset=['message'], inplace=True)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [49]:
df_fb_tag.rename(columns = {"message":"text"}, inplace=True)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [51]:
df_fb_tag.loc[df_fb_tag['valence']==2, "label"]=1

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [53]:
df_fb_tag.dropna(subset=["label"], inplace=True)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [55]:
df_fb_tag = df_fb_tag[["text", "label"]]

In [56]:
df_fb_tag

,text,label
0,gleich an die wand stellen und erschiessen..,1.0
2,Ihr seids empfindlich . . Is beste wos uns pas...,1.0
4,"Mei bitte, sie sollen sich alle gegenseitig ab...",1.0
6,"Unglaublich, was für ausländerfeindliche Komme...",1.0
10,jagt dieses Dreckspack raus.,1.0
...,...,...
1319,diese kleinen drecksäcke ..nicht genug das sie...,1.0
1321,"Dumm,dümmer ,Regierung!Die regierenden könnten...",1.0
1323,Der deutsche ist so Sau dumm das ers genau so ...,1.0
1325,Bald ficken wir ausländer euch richtig saftig,1.0


## German hasoc

In [58]:
data_path = os.path.join(dataset_path, "german",  "german_hasoc")

In [60]:
df_hasoc_1 = pd.read_csv(os.path.join(data_path, "german_dataset.tsv"), sep="\t")

In [59]:
df_hasoc_2 = pd.read_csv(os.path.join(data_path, "hasoc_de_test_gold.tsv"), sep="\t")

In [62]:
df_hasoc_1["task_1"].value_counts()

NOT    3412
HOF     407
Name: task_1, dtype: int64

In [64]:
df_hasoc_2["task_1"].value_counts()

NOT    714
HOF    136
Name: task_1, dtype: int64

In [66]:
df_hasoc = pd.concat([df_hasoc_1, df_hasoc_2])

In [69]:
df_hasoc = df_hasoc[["text", "task_1"]]

In [71]:
df_hasoc.loc[df_hasoc["task_1"]=="NOT", "label"]=0
df_hasoc.loc[df_hasoc["task_1"]=="HOF", "label"]=1

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [73]:
df_hasoc= df_hasoc[["text", "label"]]

In [75]:
df_hasoc

,text,label
0,Frank Rennicke – Ich bin stolz https://t.co/Cm...,0.0
1,ANSEHEN.....und danach bitte TEILEN...TEILEN.....,0.0
2,#Koeln Mohamed erkennt kein deutsches Recht so...,0.0
3,#SaudiArabien ist eine brutale islamische Dikt...,0.0
4,Bundespolizei #München hat im 1. Quartal 2019 ...,0.0
...,...,...
845,#Erdogan ruft seine Mops wegen Untauglichkeit ...,1.0
846,Ich würde der Linken Chefin Kipping die 1.000 ...,1.0
847,Ausländer sind Top &amp; Trumpf Schon aufgefa...,1.0
848,Wie der Regierungsjet die Regierungsflieger wi...,1.0


## IWG

In [74]:
data_path = os.path.join(dataset_path, "german",  "IWG_de")

In [79]:
df_iwg = pd.read_csv(os.path.join(data_path, "germanHatespeechRefugees.csv"))

In [82]:
df_iwg.columns

Index(['dTweet', 'HatespeechOrNot (Expert 1)', 'HatespeechOrNot (Expert 2)',
       'Hatespeech Rating (Expert 2)'],
      dtype='object')

In [84]:
df_iwg.rename(columns={"dTweet":"text", "HatespeechOrNot (Expert 1)":"anno1", "HatespeechOrNot (Expert 2)":"anno2"}, inplace=True)

In [85]:
df_iwg

,text,anno1,anno2,Hatespeech Rating (Expert 2)
0,#Rosenmontag ist abgesagt. #Rapefugees also wi...,YES,NO,5
1,bitte nicht die #Türkei zum #EU-Mitglied mache...,NO,NO,1
2,Wieso bekommen #rapefugees mehr als unsere Har...,NO,NO,3
3,Den verfluchten #Rapefugees den Krieg erklären...,NO,YES,4
4,War das Wochenende im Ruhrpott unterwegs. Über...,YES,YES,6
...,...,...,...,...
464,Wo ist das Problem? Für Moslems ist es doch sc...,YES,NO,5
465,Simone #Peter hat seit 7 Monaten und weit über...,NO,NO,1
466,Bin gespannt auf #onebillionrising. Hoffentlic...,YES,NO,3
467,#kipping ist so dumm! wenn #asylanten Wohnunge...,NO,NO,2


In [86]:
df_iwg.loc[(df_iwg["anno1"]=="YES")&(df_iwg["anno2"]=="YES"), "label"]=1
df_iwg.loc[(df_iwg["anno1"]=="NO")&(df_iwg["anno2"]=="NO"), "label"]=0


In [89]:
df_iwg = df_iwg.dropna(subset=["label"])

In [91]:
df_iwg.label.value_counts()

0.0    315
1.0     54
Name: label, dtype: int64

In [93]:
df_iwg=df_iwg[["text", "label"]]

## sharedtask 2021

In [96]:
data_path = os.path.join(dataset_path, "german",  "SharedTask-2021", "Data Sets")

In [97]:
df_test= pd.read_csv(os.path.join(data_path, "GermEval21_TestData.csv"))

In [99]:
df_train = pd.read_csv(os.path.join(data_path, "GermEval21_TrainData.csv"))

In [101]:
df_germeval_2021 = pd.concat([df_test, df_train])

In [103]:
df_germeval_2021.loc[df_germeval_2021["Sub1_Toxic"]==1, "label"]=1
df_germeval_2021.loc[df_germeval_2021["Sub1_Toxic"]==0, "label"]=0

In [104]:
df_germeval_2021.label.value_counts()

0.0    2716
1.0    1472
Name: label, dtype: int64

In [106]:
df_germeval_2021 = df_germeval_2021[["comment_text", "label"]]

In [108]:
df_germeval_2021.rename(columns={"comment_text":"text"}, inplace=True)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


# concat all datas

In [126]:
save_dir = os.path.join(dataset_path, "german",  "preprocessed")

In [127]:
df = pd.concat([df_germeval_2021, df_iwg, df_hasoc, df_fb_tag,df_germeval])

In [128]:
df = df[["text", "label"]]

In [129]:
df.label.value_counts()

0.0    12726
1.0     5283
Name: label, dtype: int64

In [130]:
df["text"] = df["text"].apply(preprocessing_one_tweet)

In [131]:
df.LEN = df.text.str.len()

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_60116/2774872017.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.LEN = df.text.str.len()


In [132]:
min(df.LEN)

0

In [133]:
df_ = df[df.LEN>10]

In [134]:
df_.label.value_counts()

0.0    12547
1.0     5268
Name: label, dtype: int64

In [135]:
df_ = df_.groupby("label").sample(5268)

In [136]:
df_ = shuffle(df_)

In [137]:
train, val, test = split_data(df_)

In [138]:
train.label.value_counts()


0.0    4226
1.0    4202
Name: label, dtype: int64

In [139]:
train.to_csv(os.path.join(save_dir, "train.csv"))
val.to_csv(os.path.join(save_dir, "val.csv"))
test.to_csv(os.path.join(save_dir, "test.csv"))

# Polish

In [8]:
data_path = os.path.join(dataset_path, "polish")

In [9]:
id_text={}
for file in os.listdir(os.path.join(data_path, "crawled")):
    filepath = os.path.join(data_path, "crawled", file)
    if file.endswith(".json"):
        with open(filepath) as f:
            data = json.load(f)
        if 'data' in data:
            for tweet in data['data']:
                id_text[tweet["id"]]= tweet["text"]

In [10]:
len(id_text)

8299

In [11]:
# task1
def get_id_dict(task, dataset):
    data_dir = os.path.join(os.path.join(data_path, task))

    with open(os.path.join(data_dir, f"{dataset}_set_clean_only_ids.txt")) as f:
        tweet_ids = [x.replace('\n', '') for x in f.readlines()]

    with open(os.path.join(data_dir, f"{dataset}_set_clean_only_tags.txt")) as f:
        labels = [x.replace('\n', '') for x in f.readlines()]
    id_label_dict = dict(zip(tweet_ids, labels))
    return id_label_dict

In [12]:
test_label_dict1 = get_id_dict("task 01", "test")
train_label_dict1 = get_id_dict("task 01", "training")
test_label_dict2 = get_id_dict("task02", "test")
train_label_dict2 = get_id_dict("task02", "training")

In [13]:
labels_1 = []
labels_2 = []
for tweet_id, text in id_text.items():
    if tweet_id in test_label_dict1:
        labels_1.append(test_label_dict1[tweet_id])
        print("test 1")
    elif tweet_id in train_label_dict1:
        labels_1.append(train_label_dict1[tweet_id])
        print("train 1")
        
    if tweet_id in test_label_dict2:
        labels_2.append(test_label_dict2[tweet_id])
        print("test 2")
    
    elif tweet_id in train_label_dict2:
        labels_2.append(train_label_dict1[tweet_id])
        print("train 2")

train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1
train 2
train 1


In [14]:
len(id_text), len(labels_1), len(labels_2)


(8299, 8299, 8299)

In [15]:
df = pd.DataFrame.from_dict(id_text, orient="index")

In [16]:
df['harmful']= labels_1
df['type']= labels_2

In [17]:
df.type.value_counts()

0    7813
1     425
2      61
Name: type, dtype: int64

In [18]:
df.harmful.value_counts()

0    7813
1     486
Name: harmful, dtype: int64

In [73]:
df = df.groupby('harmful').sample(486)
df= shuffle(df)

In [20]:
df.rename(columns={0:'text',"harmful":"label"}, inplace=True)

In [22]:
df.text = df.text.apply(preprocessing_one_tweet)

In [23]:
df=df.drop_duplicates(subset=["text"])

In [24]:
df.label.value_counts()

0    7813
1     486
Name: label, dtype: int64

In [81]:
train, val, test = split_data(df)

In [82]:
train.label.value_counts()

0    390
1    387
Name: label, dtype: int64

In [83]:
train.to_csv(os.path.join(data_path, "preprocessed", "train.csv"))
test.to_csv(os.path.join(data_path, "preprocessed", "test.csv"))
val.to_csv(os.path.join(data_path, "preprocessed", "val.csv"))


# Greek

In [25]:
data_path = os.path.join(dataset_path, "greek")

In [26]:
test_df = pd.read_csv("datasets/hate_speech_detection/greek/offenseval2020-greek/offenseval-gr-testsetv1/offenseval-gr-test-v1.tsv", sep="\t")

In [27]:
test_label = pd.read_csv("datasets/hate_speech_detection/greek/offenseval2020-greek/offenseval-gr-testsetv1/offenseval-gr-labela-v1.csv", header=None)

In [28]:
test_label.columns = ["id", "hs"]

In [29]:
test = pd.merge(test_df, test_label, on="id")

In [30]:
test

,id,tweet,hs
0,2707,@USER Θέλω να των δω από εδώ και εμπρός αν δεν...,NOT
1,2251,#survivorgr Α Και 60 φορές και με διαφορετικού...,NOT
2,9814,Και μου έλεγε η γυναίκα μου το πρωί πάρε την τ...,NOT
3,8949,κατω τα χερια απο τον #κυρανακης,NOT
4,6913,"@USER μην μας το παιζεις πονοψυχη,κρυφορατσιστ...",NOT
...,...,...,...
1539,3207,Τρελό γέλιο και σήμερα!#giatinparea,NOT
1540,6066,"Η ΚΑΤΙΑ ΓΙΑΤΙ ΚΛΑΙΕΙ, ΧΘΕΣ ΕΛΕΓΕ ΟΤΙ ΑΠΑΓΟΡΕΥΕ...",NOT
1541,2679,Γκαελ κατσε να σε ξεματιασω αγορι μου αυτη η π...,OFF
1542,3801,Εδω μου ερχεται το γνωστο hashtag της @USER #μ...,NOT


In [31]:
train_df = pd.read_csv("datasets/hate_speech_detection/greek/offenseval2020-greek/offenseval-gr-training-v1/offenseval-gr-training-v1.tsv", sep='\t')

In [32]:
train_df.rename(columns={'subtask_a':'hs'}, inplace=True)

In [33]:
df = pd.concat([test, train_df])

In [34]:
df.loc[df['hs']=='NOT', 'label']=0
df.loc[df['hs']=='OFF', 'label']=1

In [35]:
df.tweet= df.tweet.str.replace("@USER", "@user")

In [36]:
df.label.value_counts()

0.0    7559
1.0    2728
Name: label, dtype: int64

In [38]:
df = df.drop_duplicates(subset=["tweet"])

In [39]:
df.label.value_counts()

0.0    7559
1.0    2728
Name: label, dtype: int64

In [36]:
df.LEN = df.tweet.str.len()

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_56786/739952370.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.LEN = df.tweet.str.len()


In [37]:
min(df.LEN)

8

In [39]:
df.rename(columns={'tweet':'text'}, inplace=True)

In [41]:
df = df[['text', 'label']]

In [44]:
df = df.groupby("label").sample(2728)
df = shuffle(df)

In [45]:
train, dev, test = split_data(df)

In [46]:
train.label.value_counts()

1.0    2219
0.0    2145
Name: label, dtype: int64

In [47]:
train.to_csv(os.path.join(data_path, "preprocessed", "train.csv"))
dev.to_csv(os.path.join(data_path, "preprocessed", "val.csv"))
test.to_csv(os.path.join(data_path, "preprocessed", "test.csv"))

# Italian

## IHSC

In [197]:
ihsc = "datasets/hate_speech_detection/italian/IHSC"

In [203]:
tags = pd.read_csv(os.path.join(ihsc, "IHSC_ids.tsv"), sep="\t")

In [204]:
tags.loc[tags["hs"]=="yes", "label"]=1
tags.loc[tags["offensiveness"]=="strong", "label"]=1
tags.loc[(tags["hs"]=="no") & (tags['offensiveness']=="no") & (tags["aggressiveness"]=="no") & (tags["irony"]=="no") & (tags["stereotype"]=="no"), "label"]=0

In [206]:
print(len(tags))
tags.dropna(subset=["label"], inplace=True)
print(len(tags))

6928
5153


In [207]:
tags.label.value_counts()

0.0    3746
1.0    1407
Name: label, dtype: int64

In [209]:
id_text={}
for file in os.listdir(os.path.join(ihsc, "crawled")):
    filepath = os.path.join(ihsc, "crawled", file)
    if file.endswith(".json"):
        with open(filepath) as f:
            data = json.load(f)["data"]
        
        for tweet in data:
            id_text[tweet["id"]]= tweet["text"]

In [210]:
len(id_text)

5241

In [213]:
df_ihsc = pd.DataFrame.from_dict(id_text, orient="index")

In [217]:
df_ihsc.rename(columns={0:"text"}, inplace=True)

In [219]:
tag_dict=dict(zip(tags['tweet_id'], tags["label"]))

In [226]:
for tweet_id, label in tag_dict.items():
    df_ihsc.loc[df_ihsc.index==str(tweet_id), "label"]=label

In [227]:
df_ihsc.label.value_counts()

0.0    2960
1.0     914
Name: label, dtype: int64

In [230]:
df_ihsc.dropna(subset=["label"], inplace=True)

In [232]:
df_ihsc.text = df_ihsc.text.apply(preprocessing_one_tweet)

In [235]:
len(df_ihsc)

3874

In [236]:
df_ihsc.LEN = df_ihsc.text.str.len()

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_53646/3296398240.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_ihsc.LEN = df_ihsc.text.str.len()


In [238]:
min(df_ihsc.LEN)

19

In [240]:
df_ihsc.to_csv(os.path.join(ihsc, "preprocessed", "all.csv"))

In [242]:
df_ihsc.label.value_counts()

0.0    2960
1.0     914
Name: label, dtype: int64

In [243]:
df_ihsc = df_ihsc.groupby("label").sample(914, random_state=1)
df_ihsc = shuffle(df_ihsc)

In [246]:
df_ihsc.to_csv(os.path.join(ihsc, "preprocessed", "balanced.csv"))

## Haspeede

In [241]:
haspeede= "datasets/hate_speech_detection/italian/haspeede"

In [304]:
save_dir = "datasets/hate_speech_detection/italian/preprocessed"

In [252]:
df_ls=[]
for file in Path(os.path.join(haspeede, "2018")).rglob("*.tsv"):
    df = pd.read_csv(file, sep="\t", header=None)
    df_ls.append(df)

In [253]:
df_has = pd.concat(df_ls)

In [255]:
df_has = df_has[[1,2]]

In [257]:
df_has.rename(columns={1:"text", 2:"label"}, inplace=True)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [258]:
df_has

,text,label
0,Ma....anche no!,0.0
1,Ma dove vivono ?,0.0
2,Le vai a impollinare tu le piante e gli alberi...,0.0
3,Ma manda li a quel paese questi zingari bugiardi,1.0
4,"Complimenti a chi sostiene ancora questa ""poli...",1.0
...,...,...
2995,"Crescere a Cupa Perillo, il campo rom dentro S...",0.0
2996,Prima mi manda l'ufficio immigrazione in alber...,0.0
2997,@BottaAdv ... E che gli immigrati ci pagano la...,0.0
2998,Intervista choc a imam: “Roma è già musulmana....,0.0


In [259]:
df_has.label.value_counts()

0.0    4645
1.0    3355
Name: label, dtype: int64

In [260]:
path_list = ["datasets/hate_speech_detection/italian/haspeede/2020/haspeede2_dev/haspeede2_dev_taskAB.tsv", 
             "datasets/hate_speech_detection/italian/haspeede/2020/haspeede2_reference/haspeede2_reference_taskAB-tweets.tsv", 
             "datasets/hate_speech_detection/italian/haspeede/2020/haspeede2_test/haspeede2_test_taskAB-tweets.tsv"]


In [261]:
df_has_0 = pd.read_csv(path_list[0], sep="\t")

In [275]:
df_has_0.rename(columns={"text ": "text"}, inplace=True)

In [269]:
df_has_2

,id,text,hs,stereotype
0,11834,@user A me pare una scelta politica suicida pu...,NaN,NaN
1,12113,@user e' un perfetto musulmano!!! chi dice il ...,NaN,NaN
2,11770,Mai Tg e i giornaloni hanno parlato di questa ...,NaN,NaN
3,11937,@user Ipocriti farabutti. Fanno morire i terre...,NaN,NaN
4,11870,@user @user @user L'IMMIGRAZIONE C'E' STATO UN...,NaN,NaN
...,...,...,...,...
1258,10091,Il Razzismo gli viene inculcato sin da bambini...,NaN,NaN
1259,10046,PERCHÉ DI QUESTO ORRENDO STUPRO NON DI PARLA? ...,NaN,NaN
1260,10377,FERMIAMO L'IMPOSTORE. Forse saranno necesarie ...,NaN,NaN
1261,10199,@user @user @user @user @user @user @user @use...,NaN,NaN


In [265]:
df_has_1

,id,text,hs,stereotype
0,11834,@user A me pare una scelta politica suicida pu...,1,0
1,12113,@user e' un perfetto musulmano!!! chi dice il ...,1,1
2,11770,Mai Tg e i giornaloni hanno parlato di questa ...,1,1
3,11937,@user Ipocriti farabutti. Fanno morire i terre...,1,1
4,11870,@user @user @user L'IMMIGRAZIONE C'E' STATO UN...,1,1
...,...,...,...,...
1258,10091,Il Razzismo gli viene inculcato sin da bambini...,1,1
1259,10046,PERCHÉ DI QUESTO ORRENDO STUPRO NON DI PARLA? ...,1,1
1260,10377,FERMIAMO L'IMPOSTORE. Forse saranno necesarie ...,1,1
1261,10199,@user @user @user @user @user @user @user @use...,1,1


In [263]:
df_has_1 = pd.read_csv(path_list[1], sep="\t", names = ["id", "text", "hs", "stereotype"])

In [264]:
df_has_2 = pd.read_csv(path_list[2], sep="\t", names = ["id", "text", "hs", "stereotype"])

In [276]:
df_has_ = pd.concat([df_has_0, df_has_1, df_has_2])

In [278]:
df_has_.dropna(subset=["hs"], inplace=True)

In [279]:
df_has_.hs.value_counts()

0.0    4712
1.0    3388
Name: hs, dtype: int64

In [280]:
df_has_.loc[df_has_['hs']==1, "label"]=1
df_has_.loc[df_has_['hs']==0, "label"]=0

In [283]:
df_has_ = df_has_[["text", "label"]]

In [284]:
df_ = pd.concat([df_has_, df_has])

In [286]:
df_.text = df_.text.apply(preprocessing_one_tweet)

In [287]:
df_.text = df_.text.str.replace("URL", "")

In [290]:
df_.LEN = df_.text.str.len()

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_53646/3664999079.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_.LEN = df_.text.str.len()


In [292]:
df_ = df_[df_.LEN>3]

In [295]:
df = pd.concat([df_ihsc, df_])

In [297]:
df.label.value_counts()

0.0    10219
1.0     7656
Name: label, dtype: int64

In [298]:
df = df.groupby("label").sample(7656)

In [299]:
df = shuffle(df)

In [300]:
train, val, test = split_data(df)

In [303]:
test.label.value_counts()

0.0    772
1.0    760
Name: label, dtype: int64

In [305]:
train.to_csv(os.path.join(save_dir , "train.csv"), index=False)
val.to_csv(os.path.join(save_dir , "val.csv"), index=False)
test.to_csv(os.path.join(save_dir , "test.csv"), index=False)


# Dutch

In [168]:
dalc_data_path = os.path.join(dataset_path, "dutch", "DALC")

In [177]:
df_ls = []
for file in os.listdir(dalc_data_path):
    
    if file.endswith(".csv"):
        print(file)
        filepath = os.path.join(dalc_data_path, file)
        df_ = pd.read_csv(filepath, sep="\t")
        df_ls.append(df_)
        print(df_.columns)
        # print(df['Explicitness'].value_counts())

dalc_v1_dev_text.csv
Index(['Id', 'Text', 'Explicitness', 'Target'], dtype='object')
dalc_v1_test_text.csv
Index(['Id', 'Text', 'Explicitness', 'Target'], dtype='object')
dalc_v1_train_text.csv
Index(['Id', 'Text', 'Explicitness', 'Target'], dtype='object')


In [178]:
df = pd.concat(df_ls)

In [179]:
df['Text'] = df['Text'].str.replace("URL", "")
df['Text'] = df['Text'].str.replace("@USER", "@user")
df['Text'] = df['Text'].str.replace("RT", "")

In [180]:
df = df.drop_duplicates(subset=['Text'])
df= df.dropna(subset=['Text'])

In [182]:
df['LEN'] = df.Text.str.len()
df = df[df['LEN']>2]

In [183]:
df.loc[df['Explicitness']=="NOT", "label"]= 0 # normal
df.loc[df['Explicitness']=="EXPLICIT", "label"] = 1 # hate
df.loc[df['Explicitness']=="IMPLICIT", "label"] = 1 # hate

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [184]:
df.label.value_counts()

0.0    5901
1.0    1759
Name: label, dtype: int64

In [185]:
df = df.groupby('label').sample(1759)

In [186]:
df = shuffle(df)

In [187]:
df.label.value_counts()

0.0    1759
1.0    1759
Name: label, dtype: int64

In [188]:
df = df[['Text', 'label']]

In [189]:
df.rename(columns={'Text':'text'}, inplace=True)

In [190]:
len(df)

3518

In [191]:
train, val, test = split_data(df)

In [192]:
train.label.value_counts()

0.0    1419
1.0    1395
Name: label, dtype: int64

In [193]:
test.label.value_counts()

1.0    181
0.0    171
Name: label, dtype: int64

In [194]:
train.to_csv( os.path.join(dataset_path, "dutch", "train.csv"))
val.to_csv( os.path.join(dataset_path, "dutch", "val.csv"))
test.to_csv( os.path.join(dataset_path, "dutch", "test.csv"))

# French

In [8]:
# datasts hate_speech_mlma for fr

In [7]:
fr_mlma_dataset = os.path.join(dataset_path, "hate_speech_mlma_fr", "fr_dataset.csv")

In [8]:
df = pd.read_csv(fr_mlma_dataset)

In [9]:
df.sentiment.value_counts()

offensive                               1336
normal                                   821
abusive                                  594
fearful                                  236
hateful                                  207
                                        ... 
abusive_hateful_normal                     1
abusive_hateful_disrespectful_normal       1
hateful_disrespectful_normal               1
abusive_disrespectful_hateful_normal       1
disrespectful_hateful                      1
Name: sentiment, Length: 69, dtype: int64

In [11]:
df

,HITId,tweet,sentiment,directness,annotator_sentiment,target,group
0,1,rt @user personnage ainsi q tte la clique gauc...,abusive,direct,shock,origin,other
1,2,@user @user @user bah oui t'as raison l'autre ...,offensive,indirect,shock,other,individual
2,3,moi j'ai personne à qui m'adresser pour réclam...,hateful,direct,anger_confusion_sadness_fear_disgust,other,women
3,4,@user @user c...est vrai que les anchois sont ...,normal,indirect,indifference,other,other
4,5,eh les renois faut se réveiller la @url,normal,direct,sadness,origin,african_descent
...,...,...,...,...,...,...,...
4009,4010,@user tres bien. le cinema francais c...est so...,offensive,indirect,sadness,other,left_wing_people
4010,4011,@user j’ai que des rebeus en demande d’amis aussi,disrespectful_normal,direct,indifference,origin,arabs
4011,4012,@user sale juif de merde,offensive,direct,fear_shock_indifference_sadness_disgust,religion,jews
4012,4013,@user pourquoi tant de migrants africains en e...,normal,indirect,indifference,origin,immigrants


In [10]:
len(df)

4014

In [13]:
df = df.drop_duplicates(subset=["tweet"])

In [14]:
df.loc[df['sentiment']=="normal", "label"]=0 # non-hate, normal
# df.loc[df['sentiment']=="abusive", "label"]=1
df.loc[df['sentiment']=="offensive", "label"]=1
df.loc[df['sentiment']=="hateful", "label"]=1


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
df.label.value_counts()

1.0    1538
0.0     819
Name: label, dtype: int64

In [37]:
df.label.value_counts()

1.0    1543
0.0     821
Name: label, dtype: int64

In [16]:
df = df.dropna(subset=['label'])

In [17]:
len(df)

2357

In [18]:
df.tweet = df.tweet.str.replace("@url", "")

In [19]:
df=df[['tweet', 'label']]

In [20]:
df.rename(columns= {'tweet':'text'}, inplace=True)

In [23]:
df.label.value_counts()

1.0    1538
0.0     819
Name: label, dtype: int64

In [25]:
df_b = df.groupby("label").sample(819)

In [26]:
df_b.label.value_counts()

0.0    819
1.0    819
Name: label, dtype: int64

In [ ]:
# datasts conan for fr

In [27]:
df_conan=pd.read_csv( os.path.join(dataset_path, "CONAN", "CONAN", "CONAN.csv"))

In [28]:
df_conan= df_conan[df_conan['cn_id'].str.startswith("FR")==True]

In [29]:
df_conan.head()

,cn_id,hateSpeech,counterSpeech,hsType,hsSubType,cnType,age,gender,educationLevel
3864,FRT1ST0001HS0017CN000002,On ne peut pas faire confiance aux musulmans c...,Ce genre de tweet est une fraude pour l'humanité.,Islamophobia,crimes,denouncing,31.0,female,PhD
3865,FRT1ST0001HS0017CN000002P1,Les musulmans ne sont pas dignes de confiance.,Ce genre de tweet est une fraude pour l'humanité.,Islamophobia,crimes,denouncing,31.0,female,PhD
3866,FRT1ST0001HS0017CN000002P2,Vous ne devriez pas croire les musulmans.,Ce genre de tweet est une fraude pour l'humanité.,Islamophobia,crimes,denouncing,31.0,female,PhD
3867,FRT1ST0001HS0017CN000002T1,"We cannot trust the Muslims, they are all swin...",This kind of tweet is a fraud for humanity.,Islamophobia,crimes,denouncing,NaN,NaN,NaN
3868,FRT1ST0001HS0017CN000003,On ne peut pas faire confiance aux musulmans c...,Essentialisation de personnes en raison de leu...,Islamophobia,crimes,denouncing,39.0,male,Master


In [30]:
# code mix. identify language first.
import langid

def id_lang(text):
    return langid.classify(text)[0]

In [31]:
df_conan['lang_hatespeech']= df_conan['hateSpeech'].apply(id_lang)

In [32]:
df_conan['lang_counter']= df_conan['counterSpeech'].apply(id_lang)

In [33]:
len(df_conan)

6840

In [34]:
df_conan = df_conan[df_conan['lang_hatespeech']=='fr']
len(df_conan)

4854

In [35]:
df_conan= df_conan[df_conan['lang_counter']=='fr']

In [36]:
len(df_conan)

4704

In [37]:
df_conan_hs =pd.DataFrame({"text": df_conan['hateSpeech'], "label":[1 for x in range(len(df_conan['hateSpeech']))]})

In [38]:
df_conan_normal =pd.DataFrame({"text": df_conan['counterSpeech'], "label":[0 for x in range(len(df_conan['counterSpeech']))]})

In [39]:
df_conan_ =shuffle( pd.concat([df_conan_hs, df_conan_normal]))

In [40]:
len(df_conan_)

9408

In [45]:
df_conan_ = df_conan_.drop_duplicates(subset=["text"])

In [46]:
len(df_conan_)

1789

In [47]:
df_conan_.label.value_counts()

0    1643
1     146
Name: label, dtype: int64

In [48]:
df = pd.concat([df_conan_, df_b])


In [49]:
df.label.value_counts()

0.0    2462
1.0     965
Name: label, dtype: int64

In [52]:
df=df.groupby("label").sample(965)

In [53]:
df = shuffle(df)

In [55]:
df.label.value_counts()

1.0    965
0.0    965
Name: label, dtype: int64

In [56]:
train, val, test = split_data(df)

In [57]:
train.label.value_counts()

1.0    779
0.0    765
Name: label, dtype: int64

In [58]:
val.label.value_counts()

0.0    98
1.0    95
Name: label, dtype: int64

In [59]:
test.label.value_counts()

0.0    102
1.0     91
Name: label, dtype: int64

In [60]:
train.to_csv(os.path.join(dataset_path, "french", "preprocessed", "train.csv"))
test.to_csv(os.path.join(dataset_path, "french", "preprocessed", "test.csv"))
val.to_csv(os.path.join(dataset_path, "french", "preprocessed", "val.csv"))